# 반드시 처음부터 끝까지 스켈레톤 코드를 살펴보고 구현하기 시작하길 바란다
## 1. 스켈레톤 코드를 [복사 및 편집] 하여 사용한다.
## 2. 아래의 [Empty Module 5개]를 직접 구현한다.
## 3. main 부를 실행하면 여러분이 작성한 Empty Module 을 이용하여 코드가 동작한다.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ml-p12/Label2Names.csv
/kaggle/input/2021-ml-p12/submission.csv
/kaggle/input/2021-ml-p12/train_csv_v2/scorpion/image_0080.csv
/kaggle/input/2021-ml-p12/train_csv_v2/scorpion/image_0015.csv
/kaggle/input/2021-ml-p12/train_csv_v2/scorpion/image_0066.csv
/kaggle/input/2021-ml-p12/train_csv_v2/scorpion/image_0049.csv
/kaggle/input/2021-ml-p12/train_csv_v2/scorpion/image_0063.csv
/kaggle/input/2021-ml-p12/train_csv_v2/scorpion/image_0020.csv
/kaggle/input/2021-ml-p12/train_csv_v2/scorpion/image_0028.csv
/kaggle/input/2021-ml-p12/train_csv_v2/scorpion/image_0014.csv
/kaggle/input/2021-ml-p12/train_csv_v2/scorpion/image_0054.csv
/kaggle/input/2021-ml-p12/train_csv_v2/scorpion/image_0004.csv
/kaggle/input/2021-ml-p12/train_csv_v2/scorpion/image_0009.csv
/kaggle/input/2021-ml-p12/train_csv_v2/scorpion/image_0021.csv
/kaggle/input/2021-ml-p12/train_csv_v2/scorpion/image_0070.csv
/kaggle/input/2021-ml-p12/train_csv_v2/scorpion/image_0046.csv
/kaggle/input/2021-ml-p12/train_csv

## [라이브러리 설치] 컴퓨터 비전의 대표 라이브러리 [OpenCV](https://opencv.org/) 
Opencv는 다양한 영상/동영상 처리에 사용할 수 있는 오픈소스 라이브러리. 이미지 데이터를 처리하기 위한 다양한 함수를 제공하고 있으며, 본 실습에서는 영상의 칼라 변환(RGB영상을 회색영상 변환)시, 특징량 추출(SIFT)시 사용됩니다. OpenCV는 C++로 구현되어 있으며, 파이썬 Wrapper를 제공하고 있어 파이썬에서도 사용가능합니다. JupyterNotebook 환경에서는 import cv2로 불러 와서 사용할 수 있습니다. 현재 Kaggle 노트북 개발 환경에 설치된 OpenCV 라이브러리로는 우리의 실습를 원활하게 수행할 수 없으므로, 아래와 같이 기조에 설치된 OpenCV 라이브러리를 제거하고 우리에게 맞는 버전을 설치합니다.

In [2]:
# 현재 opencv 코드는 3.4.2.17 에 맞춰서 작성되어 있어 버전을 맞추는 작업이 필요하다. 
! yes | pip3 uninstall opencv-python
! yes | pip3 uninstall opencv-contrib-python
! yes | pip3 install opencv-python==3.4.2.17
! yes | pip3 install opencv-contrib-python==3.4.2.17
!pip install faiss-gpu

Found existing installation: opencv-python 4.5.1.48
Uninstalling opencv-python-4.5.1.48:
  Would remove:
    /opt/conda/lib/python3.7/site-packages/cv2/*
    /opt/conda/lib/python3.7/site-packages/opencv_python-4.5.1.48.dist-info/*
    /opt/conda/lib/python3.7/site-packages/opencv_python.libs/libQt5Concurrent-21c64331.so.5.15.0
    /opt/conda/lib/python3.7/site-packages/opencv_python.libs/libQt5Core-d92f2856.so.5.15.0
    /opt/conda/lib/python3.7/site-packages/opencv_python.libs/libQt5Gui-ca084835.so.5.15.0
    /opt/conda/lib/python3.7/site-packages/opencv_python.libs/libQt5Test-9d4ee57e.so.5.15.0
    /opt/conda/lib/python3.7/site-packages/opencv_python.libs/libQt5Widgets-0e1d98b5.so.5.15.0
    /opt/conda/lib/python3.7/site-packages/opencv_python.libs/libQt5XcbQpa-70670cdb.so.5.15.0
    /opt/conda/lib/python3.7/site-packages/opencv_python.libs/libX11-xcb-1ae4fd0e.so.1.0.0
    /opt/conda/lib/python3.7/site-packages/opencv_python.libs/libXau-00ec42fe.so.6.0.0
    /opt/conda/lib/python3.7

In [3]:
import cv2
cv2.__version__

'3.4.2'

In [4]:
# OpenCV를 사용하기 위해 Import한다.
import cv2

import os
import numpy as np
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn import svm
import pandas as pd
from tqdm import tqdm
from scipy.cluster.vq import vq

# >> 필요한 모듈 부분 (function 부)

# [Empty Module #1] 데이터 불러오기 

### 목표: 지금부터 1D 데이터 형테로 변환하여 저장되어 있는 csv 파일을 읽고, 학습/테스트 데이터를 변수(메모리)에 저장하는 코드를 작성하기로 한다.
- 1) 클래스 명을 범주형 라벨로 변환하는 작업을 진행한다. 
  - (팁) 클래스 명: 저장된 CSV파일의 상위 폴더 이름, 범주형 라벨: Label2Names.csv에 있는 매핑 표
  - (예) "crocodile_head" -> 30
  - (주의) "BACKGROUND_Google" 는 Label2Names.csv 안에 없므며, 배경클래스는 102로 맵핑하여 사용하라
- 2) 특징점을 추출하기 위해서는 1D 데이터를 2D 형태인 RGB 데이터로 변환이 필요하다. 이와 같이 데이터의 형태를 변환할 때는 reshape 함수를 사용하라.
- 3) 그리고 uint64로 제공된 데이터를 uint8로 변경하여 사용한다. 이때 np.uint8 을 사용하라.
- 4) 특징점을 추출하는 OpenCV 함수들은 입력으로 칼라 영상이 아닌 Gray 영상을 입력으로 받으므로 영상 칼라를 변환한다. 
 - 현재 제공된 영상 데이터는 칼라 포멧으로 BGR 포맷을 사용하고 있다. (일반적으로 칼라 영상은 RGB 이다.)
 - 이때 [cv2.cvtcolor() 라는 함수](https://opencv-python.readthedocs.io/en/latest/doc/08.imageProcessing/imageProcessing.html)를 사용하고, flag로 cv2.COLOR_BGR2GRAY 를 사용하라. 

In [50]:
# -------------------------------------
# [Empty Module #1] 학습데이터, 테스트 데이터 불러오기 
# -------------------------------------


# -------------------------------------
# loadtrain(dataroottrain): <= 코드를 추가하여 학습데이터를 불러오는 코드를 완성하세요
# -------------------------------------
# 목적: 학습데이터 불러오기
# 입력인자: dataroottrain - 학습 데이터 경로
# 출력인자: traindata - 학습데이터(2D img)
#          trainlabel - 학습데이터의 라벨 
# -------------------------------------

def loadtrain(dataroottrain):
    trainlabel=[]
    traindata=[]
    for classname in tqdm(os.listdir(dataroottrain)):
        # ------------------------------------------------------------
        # 구현 가이드라인  (1)
        # ------------------------------------------------------------
        # if classname 이 "BACKGROUND_Google" 이라면 
        #   trainlabel에 102가 현재 클래스의 Index
        # else
        #   classname 을 df_data 속에서 찾은 다음 그에 해당하는 index가 현재 클래스의 Index 
        # ------------------------------------------------------------
        # 구현 가이드라인을 참고하여 아래쪽에 코드를 추가하라
        if classname == 'BACKGROUND_Google':
            index=102
        else:
            index=np.int(df_data[0].loc[df_data[1] == classname])
            
        for csvname in os.listdir(os.path.join(dataroottrain,classname)):
            # ------------------------------------------------------------
            # 구현 가이드라인 (2)
            # ------------------------------------------------------------
            # 1) trainlabel에 현재 클래스 index를 append
            # 2) csvpath= dataroottrain -> classname -> csvname Tip) os.path.join을 사용해 csvpath 설정 
            # 3) pd.read_csv를 이용해 csv 읽어온후 np.array를 사용해 numpy 변수로 변환
            # 4) csv를 읽으면 matrix는 현재 [256*256*3,] 차원임 이를 [256,256,3] 으로 변경 Tip) np.reshape 함수 사용
            # 5) np.uint8을 이용해 타입변경
            # 6) cv2.cvtColor 를 사용해 영상의 차원을 변경 Tip:)cv2.COLOR_BGR2GRAY를 인자로 사용 
            # 7) traindata에 변경된 데이터 append
            # ------------------------------------------------------------
            # 구현 가이드라인을 참고하여 아래쪽에 코드를 추가하라
            trainlabel.append(index)
            csvpath = os.path.join(dataroottrain,classname,csvname)
            data = np.array(pd.read_csv(csvpath))
            data = np.reshape(data,(256,256,3))
            data = np.uint8(data)
            data= cv2.cvtColor(data,cv2.COLOR_BGR2GRAY)
            traindata.append(data)

    return traindata,trainlabel



# -------------------------------------
# loadtest(dataroottest): <= 코드를 추가하여 데스트 데이터를 불러오는 코드를 완성하세요
# -------------------------------------
# 목적: 테스트데이터 불러오기
# 입력인자: dataroottest - 테스트 데이터 경로
# 출력인자: testdata - 테스트데이터(2D img)
#          testlabel - 테스트데이터의 파일 제목  
# -------------------------------------
def loadtest(dataroottest):
    testlabel=[]
    testdata=[]
    testsort=sorted(os.listdir(dataroottest))
    
    for csvname in tqdm(testsort):
        # ------------------------------------------------------------
        # 구현 가이드라인
        # ------------------------------------------------------------
        # 1) csvpath= dataroottest -> csvname Tip) os.path.join을 사용해 csvpath 설정 
        # 2) pd.read_csv를 이용해 csv 읽어온후 np.array를 사용해 numpy 변수로 변환
        # 3) csv를 읽으면 matrix는 현재 [256*256*3,] 차원임 이를 [256,256,3] 으로 변경 Tip) np.reshape 함수 사용
        # 4) np.uint8을 이용해 타입변경
        # 5) cv2.cvtColor 를 사용해 영상의 차원을 변경 Tip:)cv2.COLOR_BGR2GRAY를 인자로 사용 
        # 6) testdata에 변경된 데이터 append
        # ------------------------------------------------------------
        # 구현 가이드라인을 참고하여 아래쪽에 코드를 추가하라   
        
        testlabel.append(csvname)
        csvpath = os.path.join(dataroottest,csvname)
        data = np.array(pd.read_csv(csvpath))
        data = np.reshape(data,(256,256,3))
        data = np.uint8(data)
        data = cv2.cvtColor(data,cv2.COLOR_BGR2GRAY)
        testdata.append(data)
        
        
    return testdata,testlabel


# [Empty Module #2] 특징점 추출하기 (Dense-SIFT)
- ```extract_sift_descriptors``` 함수를 바탕으로 ```extract_DenseSift_descriptors``` 함수 작성
- SIFT의 detect 함수와 compute 함수를 활용한다.
- SIFT의 detect 함수는 특징점 위치를 추출하는 함수이고, compute 함수는 특징점 위치 주변 정보를 추출하는 함수이다.
- [참고 메뉴얼](https://docs.opencv.org/3.4/d0/d13/classcv_1_1Feature2D.html#ab3cce8d56f4fc5e1d530b5931e1e8dc0) 참고  
    

In [6]:
# -------------------------------------
# [Empty Module #2] 특징점 추출하기 (Dense-SIFT 사용)
# -------------------------------------


# -------------------------------------
# extract_sift_descriptors(img): <= 수정하지 말고 Dense-SIFT를 위해 참고하세요
# -------------------------------------
# 목적: SIFT 특징점 추출을 위한 함수
# 입력인자: img - 2D 흑백 영상 [256,256]
# 출력인자: descriptors - SIFT 기술자
# -------------------------------------
def extract_sift_descriptors(img):
    
    # SIFT 인스턴스 생성
    sift = cv2.xfeatures2d.SIFT_create()
    # SIFT를 이용하여 특징점 위치 추출
    keypoints = sift.detect(img,None)
    # img의 type을 uint8로 변환 
    img=np.uint8(img)
    # SIFT를 이용하여 추출된 위치 주변 정보인 기술자 추출
    _,descriptors = sift.compute(img,keypoints)
    # SIFT 기술자 반환
    return descriptors


# -------------------------------------
# extract_DenseSift_descriptors(img): <= 코드를 추가하여 Dense-SIFT를 추출하는 코드를 완성하세요
# -------------------------------------
# 목적: Dense-SIFT 특징점 추출을 위한 함수
# 입력인자: img - 2D 흑백 영상 [256,256]
# 출력인자: descriptors - Dense-SIFT 기술자
# -------------------------------------
def extract_DenseSift_descriptors(img):

    # ------------------------------------------------------------
    # 1) SIFT 인스턴스 생성
    # ------------------------------------------------------------
    # 구현 가이드라인을 참고하여 아래쪽에 코드를 추가하라
    sift = cv2.xfeatures2d.SIFT_create()

    
    
    # SIFT 기술자를 추출하고 싶은 특징점 위치 정의하기
    keypoints = [cv2.KeyPoint(x, y, DSIFT_STEP_SIZE)
            for y in range(0, img.shape[0], DSIFT_STEP_SIZE)
                for x in range(0, img.shape[1], DSIFT_STEP_SIZE)]
    

    # ------------------------------------------------------------
    # 2) SIFT를 이용하여 Dense SIFT를 추출하시오. (단, Keypoints는 Dense-SIFT의 사전정의된 위치 정보를 담고 있음)
    # ------------------------------------------------------------
    # 구현 가이드라인을 참고하여 아래쪽에 코드를 추가하라
    img=np.uint8(img)
    _,descriptors = sift.compute(img,keypoints)

    
    
    
    return descriptors


## [Empty Module #3] Codebook 생성

- 1) 추출한 Dense-SIFT의 특징량(기술자)을 이용하여 Codebook 을 생성한다
  - (팁) sckikit-learn에 있는 Kmeans 사용할 것 [메뉴얼](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)
  - 1) X를 [3060 * 1024,128]로 reshape 할 것 **
  - 2) Kmeans의 Cluster 갯수(voc_size)를 입력으로 Kmeans를 정의한 후
  - 3) X(data) 를 fit
  - kmeans center를 return 
  -  ***유의 사항* ** 원래 두시간 넘게 걸리는 작업이므로 이상하게 생각안해도 된다. 


In [7]:
# ------------------------------------------------------------
# [Empty Module #3] 코드북 생성하기 (KMeans 사용)
# ------------------------------------------------------------



# ------------------------------------------------------------
# build_codebook(X, voc_size): <= 코드를 추가하여 build_codebook를 만드는 코드를 완성하세요
# ------------------------------------------------------------
# 목적: Visual Words (즉, codebook)을 계산하기 위한 함수
#
# 입력인자: X - 군집화 하려는 데이터 [3060,1024,128]
#         voc_size - 군집화 하려는 센터(Cluster) 수, 즉 Kmeans의 K
#
# 출력인자:  Kmeans 내장 변수인 cluster_center를 반환
# ------------------------------------------------------------
def build_codebook(X, voc_size):
    # ------------------------------------------------------------
    # 구현 가이드라인 
    # ------------------------------------------------------------
    # 1) X는 [3060*1024,128]의 형태로 reshape 한다 **
    # 2) n_clusters를 voc_size로 주고 KMeans를 정의한다
    # 3) KMeans의 fit 함수를 사용해 data의 cluster-center를 구한다
    # ------------------------------------------------------------
    # 구현 가이드라인을 참고하여 아래쪽에 코드를 추가하라
    #X = np.reshape(X,(3060*1024,128))
    #model=KMeans(n_clusters = voc_size)
    #model.fit(X)
    import faiss
    feature=np.array(X).reshape(-1,128).astype('float32')
    d=feature.shape[1]
    k=voc_size
    clus = faiss.Clustering(d, k)
    clus.niter = 300
    clus.seed =10
    clus.max_points_per_centroid = 10000000
    ngpu=1
    res = [faiss.StandardGpuResources() for i in range(ngpu)]
    flat_config = []
    for i in range(ngpu):
        cfg = faiss.GpuIndexFlatConfig()
        cfg.useFloat16 = False
        cfg.device = i
        flat_config.append(cfg)
        if ngpu == 1:
            index = faiss.GpuIndexFlatL2(res[0], d, flat_config[0])
            clus.train(feature, index)
            centroids = faiss.vector_float_to_array(clus.centroids)
            centroids=centroids.reshape(k, d)
    
    return centroids # xxx 는 작성자가 변경해야 함


## [Empty Module #4] BoW: histogram vector 생성

### BoW
- 생성해둔 codebook(kmeans clustrer center) 와 Dense SIFT를 비교하여 histogram을 구하는 작업 진행
- 1) scipy.cluster.vq를 사용해 각 Dense SIFT와 유사한 codebook의 Index를 반환 받아 사용. 
  - (팁) [메뉴얼](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.vq.vq.html) 참고
- 2) vq를 이용해 얻은 Index를 np.histogram 을 사용해 histogram을 구한다.
  - (팁)np.histogram 사용시 bin=range(codebook.shape[0] + 1), normed=True 설정
  - [메뉴얼](https://numpy.org/doc/stable/reference/generated/numpy.histogram.html) 참고



In [8]:
# ------------------------------------------------------------
# [Empty Module #4] 히스토그램 생성하기 (vq,np.histogram 사용)
# ------------------------------------------------------------

# ------------------------------------------------------------
# BoW(descriptor, codebook): <= 코드를 추가하여 histogram을 만드는 코드를 완성하세요
# ------------------------------------------------------------
# 목적: Histogram (즉, 빈도수)를 계산하기 위한 함수
#
# 입력인자: descriptor - 한 영상의 특징점 Matrix [1024,128] 
#         codebook - 비교하기 위한 Visual word [200,128]
#
# 출력인자:  feature의 codebook 빈도수( 즉 histogram)을 flatten한 Matrix

# ------------------------------------------------------------
def BoW( descriptor, codebook):
    descriptor=descriptor.tolist()
    # ------------------------------------------------------------
    # 구현 가이드라인 
    # ------------------------------------------------------------
    # 1) scipy.cluster.vq 를 사용해 각 Dense SIFT(descriptor)와 유사한 codebook의 Index를 반환 받아 사용
    # 2) 구한 Index들 histogram을  np.histogram 을 사용해 histogram을 구한다
    # 2-Tip) np.histogram 사용시 bins=range(codebook.shape[0] + 1), normed=True 로 설정
    # ------------------------------------------------------------
    # 구현 가이드라인을 참고하여 아래쪽에 코드를 추가하라
    data =vq(descriptor, codebook)
    hist, bins = np.histogram(data,bins=range(codebook.shape[0] + 1), normed=True)

    
    return hist.flatten() # xxx는 작성자가 변경해야 함


## [Empty Module #5] SVM: classifier

- svm.SVC() 를 활용해 classification 을 진행
- GridSearchCV()를 활용해 SVC의 파라미터 튜닝을 진행 .
  - (팁) parameters = {'kernel':('linear', 'rbf'), 'C':[0.1 ,1, 10, 100, 1000]}
- fit()으로 train_data에 대한 머신러닝 학습
- predict()으로 test_data에 대한 정답을 추론 하여 반환 



# **Empty Module5**

In [9]:
# ------------------------------------------------------------
# [Empty Module #5] SVM 학습 및 실험데이터 classification  (SVM 사용)
# ------------------------------------------------------------

# ------------------------------------------------------------
# SVM(train_data,train_label, test_data): <= 코드를 추가하여 test_data를 classification하는 코드를 완성하세요
# ------------------------------------------------------------
# 목적: SVM 학습 및 test_data의 class를 분류하는 함수 
#
# 입력인자: train_data - 모든 학습 영상의 BoW features(histogram) [3060,200] 
#         train_label - 모든 학습 영상의 class [3060,1]
#         test_data   - 모든 테스트 영상의 BoW features(histogram) [1712,200]
# 출력인자: test_data을 SVM으로 분류한 결과 

# ------------------------------------------------------------
def SVM(train_data,train_label,test_data):
    # ------------------------------------------------------------
    # 구현 가이드라인 
    # ------------------------------------------------------------
    # 1) svm.SVC() 를 사용해 SVM 정의 
    # 2) SVM과 파라미터 {'kernel':('linear', 'rbf'), 'C':[0.1 ,1, 10, 100, 1000]} 를 사용해 GridSearchCV() 정의
    # 3) 정의한 GridSearchCV의 함수인 fit을 사용해 train_data 학습 
    # 4) 정의한 GridSearchCV의 predict 함수를 사용해 test_data 에 대한 정답을 추론
    # ------------------------------------------------------------
    # 구현 가이드라인을 참고하여 아래쪽에 코드를 추가하라
    model = svm.SVC()
    param_grid = {'kernel':('linear', 'rbf'), 'C':[0.1 ,1, 10, 100, 1000]}
    clf = GridSearchCV(model, param_grid, cv=5)
    clf.fit(train_data,train_label)
    predict = clf.predict(test_data)
    return predict

# >> 코드 시작 부분 (main 부)

In [10]:
#현재 경로 저장 
dataroot="/kaggle/input/2021-ml-p12/"
# Class 이름과 Index가 맵핑되어 있는 csv 경로 저장 
csvpath=os.path.join(dataroot,"Label2Names.csv")
# 위 csv 경로로 csv 읽어오기 
df_data=pd.read_csv(csvpath,header=None)

# 학습 데이터 와 테스트 데이터 경로 저장 
dataroottrain=os.path.join(dataroot,"train_csv_v2")
dataroottest=os.path.join(dataroot,"test_csv_v2")

In [49]:
index=np.int(df_data[0][df_data[1] == 'Faces'])
print(index)

1


In [51]:
# CSV로 저장되어 있는 Img 데이터 읽어오기 (학습데이터, 테스트 데이터)
train_data,train_label=loadtrain(dataroottrain)
test_data,test_label=loadtest(dataroottest)   

# numpy 형식으로 변환 
train_data=np.array(train_data)
train_label=np.array(train_label)
test_data=np.array(test_data)
test_label=np.array(test_label)

100%|██████████| 1712/1712 [00:30<00:00, 57.06it/s]


In [26]:
train_label

array([81    82
Name: 0, dtype: int64, 81    82
Name: 0, dtype: int64,
       81    82
Name: 0, dtype: int64, ...,
       35    36
Name: 0, dtype: int64, 35    36
Name: 0, dtype: int64,
       35    36
Name: 0, dtype: int64], dtype=object)

In [12]:
# Dense SIFT의 Keypoint 간격 설정 
DSIFT_STEP_SIZE = 8
# 영상의 특징점(DenseSIFT) 검출(학습데이터, 테스트데이터)
x_train_feature = [extract_DenseSift_descriptors(img) for img in tqdm(train_data)]
x_test_feature = [extract_DenseSift_descriptors(img) for img in tqdm(test_data)]
# numpy 형식으로 변환
x_train_des = np.array(x_train_feature)
x_test_des = np.array(x_test_feature)

100%|██████████| 1712/1712 [02:21<00:00, 12.13it/s]


In [13]:
# num_cluster를 200으로 설졍하여 Visual Words(codebook) 계산 
codebook = build_codebook(x_train_des, 200)

In [14]:
print(np.shape(codebook))

(200, 128)


In [15]:
# BoW feature 계산 (학습 데이터, 테스트 데이터)
x_train = [BoW(x_train_des[i],codebook) for i in range(len(train_data))]
x_train = np.asarray(x_train)
x_test = [BoW(x_test_des[i],codebook) for i in range(len(test_data))]
x_test = np.asarray(x_test)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:27: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.


In [52]:
print(np.shape(x_train))
print(np.shape(train_label))
print(np.shape(x_test))
train_label

(3060, 200)
(3060,)
(1712, 200)


array([82, 82, 82, ..., 36, 36, 36])

In [53]:
# SVM으로 테스트 데이터 클래스 분류
predict=SVM(x_train,train_label,x_test) 

In [54]:
# CSV 제출을 위한 predict shape 변환 
predict=predict.reshape(-1,1)
# CSV 제출을 위한 test_label(id) shape 변환 
test_label=np.array(test_label)
test_label=test_label.reshape(-1,1)
# CSV 제출을 위한 Predict 와 label 합치기 
total_result=np.hstack([test_label,predict])
# CSV 제출을 위한 Dataframe으로 변환 
df = pd.DataFrame(total_result,columns=["Id","Category"])
# CSV 저장 
df.to_csv("./Test_2021ML.csv",index=False,header=True)

In [55]:
predict

array([[57],
       [42],
       [23],
       ...,
       [83],
       [60],
       [92]])